In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from skimage import io, transform
import scipy.misc as m
from tqdm import tqdm
import shutil
import tensorflow as tf
from tensorflow.contrib.data import Dataset, Iterator

In [2]:
# Preparing the dataset
data_dir = '/Users/navneetmkumar/Desktop/TUM/lfw'
users = os.listdir(data_dir)
print(len(users))
users = sorted(users[1:])

5750


In [3]:
root_dir = '/Users/navneetmkumar/Desktop/TUM/'
image_dir = root_dir+ 'lfw/'

if not os.path.exists(root_dir+'train-inputs'):
    os.makedirs(root_dir+'train-inputs')

if not os.path.exists(root_dir+'train-targets'):
    os.makedirs(root_dir+'train-targets')
# Getting the data into a single training folder
def create_training_set(users, root_dir, image_dir):
    for user in users[1:]:
        path = os.path.join(image_dir, user)
        for f in os.listdir(path):
            f_name = os.path.basename(f)
            f_name = f_name.split(".")[0]
            f = os.path.join(path, f)
            shutil.copy2(f, root_dir+'train-inputs')

In [ ]:
create_training_set(users, root_dir, image_dir)

In [4]:
train_dir = root_dir+'train-inputs/'
# Creating the targets dataset
def create_targets_set(train_dir):
    for f in sorted(os.listdir(train_dir)):
        f_name = os.path.join(train_dir+f)
        img = io.imread(f_name)
        # Make the transformation (Gaussian Blur)
        blur = cv2.GaussianBlur(img, (7,7), 0)
        io.imsave(root_dir+'train-targets/'+f,blur)

In [ ]:
create_targets_set(train_dir)

In [5]:
print(len(os.listdir(train_dir)))

13232


In [6]:
train_targets_dir = root_dir+'train-targets/'
print(len(os.listdir(train_targets_dir)))

13232


In [12]:
train_imgs = os.listdir(train_dir)
train_targets = os.listdir(train_targets_dir)

In [8]:
# Defining the global variables
IMG_SIZE = 250
# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     
NUM_EPOCHS_PER_DECAY = 350.0      
LEARNING_RATE_DECAY_FACTOR = 0.1  
INITIAL_LEARNING_RATE = 0.1       

# Creating the model

batch_size= 128

def GaussianFilterModel(data):
    
    number_of_classes = data.shape[2]
    
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = tf.get_variable('w1', [5,5,3,512], initializer=tf.truncated_normal_initializer(5e-2), dtype=tf.float32)
        conv = tf.nn.conv2d(data, kernel, [1,1,1,1], padding='SAME')
        biases = tf.get_variable('b1', [512], initializer=tf.constant_initializer(0.1), dtype=tf.float32)
        pre_activation = tf.nn.bias_add(conv, biases)
        act = tf.nn.relu(pre_activation, name=scope.name)
        
        #norm
        norm = tf.nn.lrn(act, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm')
        
        #pool
        pool = tf.nn.max_pool(norm, ksize=[1, 3, 3, 1],
                         strides=[1, 2, 2, 1], padding='SAME', name='pool')
        
    with tf.variable_scope('conv4') as scope:
        kernel2 = tf.get_variable('w5', [7, 7, 512, 4096], initializer=tf.truncated_normal_initializer(5e-2), dtype=tf.float32)
        conv2 = tf.nn.conv2d(pool, kernel2, [1,1,1,1], padding='SAME')
        biases2 = tf.get_variable('b5', [4096], initializer=tf.constant_initializer(0.1), dtype=tf.float32)
        pre_activation2 = tf.nn.bias_add(conv2, biases2)
        act5 = tf.nn.relu(pre_activation2, name=scope.name)
        
        kernel4 = tf.get_variable('w7', [1,1,4096, 3], initializer=tf.truncated_normal_initializer(5e-2), dtype=tf.float32)
        conv4 = tf.nn.conv2d(act5, kernel4, [1,1,1,1], padding='SAME')
        biases4 = tf.get_variable('b7', [3], initializer=tf.constant_initializer(0.1), dtype=tf.float32)
        pre_activation4 = tf.nn.bias_add(conv4, biases4)
    
    with tf.variable_scope('local1') as scope:
        output = tf.image.resize_images(pre_activation4, tf.Variable([250, 250], tf.int32))
        
    return output
        

In [9]:
#validation split
validation_size = 0.2
# Input data
x = tf.placeholder(tf.float32, shape=[None, IMG_SIZE, IMG_SIZE, 3], name='x')
y = tf.placeholder(tf.float32, shape=[None, IMG_SIZE, IMG_SIZE, 3], name='y')

In [10]:
train_images = []
train_targets = []
for f in train_imgs:
    f_name = train_dir+f
    train_images.append(f_name)

In [13]:
train_labels = []
for f in train_targets:
    f_name = train_targets_dir + f
    train_labels.append(f_name)

In [14]:
train_images = train_images[0:1000]
train_labels = train_labels[0:1000]

In [15]:
def input_parser(image_path, label_path):
    img_file = tf.read_file(image_path)
    label_file = tf.read_file(label_path)
    img_decoded = tf.image.decode_image(img_file, channels=3)
    lbl_decoded = tf.image.decode_image(label_file, channels=3)
    
    return img_decoded, lbl_decoded

In [17]:
print(tr_data.output_types) 
print(tr_data.output_shapes)  

(tf.uint8, tf.uint8)
(TensorShape(None), TensorShape(None))


In [16]:
tr_data = Dataset.from_tensor_slices((train_images, train_labels))
tr_data = tr_data.map(input_parser)
tr_data = tr_data.batch(batch_size)
# create TensorFlow Iterator object
iterator = Iterator.from_structure(tr_data.output_types,
                                   tr_data.output_shapes)
next_element = iterator.get_next()

# create two initialization ops to switch between the datasets
training_init_op = iterator.make_initializer(tr_data)

In [18]:
# Define the training 
def train_model(x, y):
    prediction = GaussianFilterModel(x)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(training_init_op)
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            while True:
                try:
                    elem = sess.run(next_element)
                    epoch_x, epoch_y = elem
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                    epoch_loss += c
                    
                except tf.errors.OutOfRangeError:
                    print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

In [ ]:
train_model(x, y)